# Segmenting and Clustering Neighborhoods in Toronto City

## Importing required libararies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!conda install -c conda-forge bs4 --yes # uncomment this line if you haven't completed the Foursquare API lab
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## Rendering the Wikipedia page for neighbourhood data and converting to a clean data frame

In [2]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
#website_url

In [3]:
soup = BeautifulSoup(website_url)
#print(soup.prettify())

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#print(My_table)

In [5]:
rows = My_table.find_all('tr')
len(rows)

181

In [6]:
columns  = [cell.get_text().replace('\n','') for cell in rows[0].find_all('th')]
columns

['Postal Code', 'Borough', 'Neighbourhood']

In [7]:
neighborhood_data = pd.DataFrame(columns=columns)
for row in rows[1:]:
    cells = [cell.get_text().replace('\n','') for cell in row.find_all('td')]
    #print(cells)
    if cells[1] == "Not assigned":
        continue
    if cells[2] == "Not assigned":
        cells[2] = cells[1]
    neighborhood_data = neighborhood_data.append({columns[0]: cells[0],
                                          columns[1]: cells[1],
                                          columns[2]: cells[2]}, ignore_index=True)


In [8]:
neighborhood_data.head(15)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [9]:
neighborhood_data.shape

(103, 3)

In [10]:
print("Number of Boroughs =",len(neighborhood_data['Borough'].unique()))
print("Number of Neighbourhoods =",neighborhood_data.shape[0])

Number of Boroughs = 10
Number of Neighbourhoods = 103


In [11]:
neighborhood_data.groupby(['Borough']).count()

,Postal Code,Neighbourhood
Borough,,
Central Toronto,9,9
Downtown Toronto,19,19
East Toronto,5,5
East York,5,5
Etobicoke,12,12
Mississauga,1,1
North York,24,24
Scarborough,17,17
West Toronto,6,6


### lets just work on the Borough with maximum Neighborhoods

In [12]:
EBC = neighborhood_data[neighborhood_data['Borough']=='North York'].reset_index(drop=True)
EBC.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M6A,North York,"Lawrence Manor, Lawrence Heights"
3,M3B,North York,Don Mills
4,M6B,North York,Glencairn
5,M3C,North York,Don Mills
6,M2H,North York,Hillcrest Village
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North"
8,M2J,North York,"Fairview, Henry Farm, Oriole"
9,M3J,North York,"Northwood Park, York University"


In [13]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
EBC = EBC.join(coordinates.set_index('Postal Code'), on='Postal Code')
EBC.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [15]:
EBC.shape

(24, 5)

In [16]:
EBC= EBC[~EBC['Latitude'].isna()]
EBC.shape

(24, 5)

In [17]:
EBC.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


#### Define Foursquare Credentials and Version


In [18]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [19]:
# create map of Etobicoke using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(EBC['Latitude'], EBC['Longitude'], EBC['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [20]:
CLIENT_ID = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxxxxxxxxxxxxxxxxxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

## Explore Neighborhoods in Etobicoke


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
# type your answer here

EBC_venues = getNearbyVenues(EBC['Neighbourhood'],EBC['Latitude'], EBC['Longitude'],  radius=1115)


Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


#### Let's check the size of the resulting dataframe


In [23]:
print(EBC_venues.shape)
EBC_venues.head()

(762, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
3,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


Let's check how many venues were returned for each neighborhood


In [24]:
EBC_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",32,32,32,32,32,32
Bayview Village,13,13,13,13,13,13
"Bedford Park, Lawrence Manor East",49,49,49,49,49,49
Don Mills,121,121,121,121,121,121
Downsview,67,67,67,67,67,67
"Fairview, Henry Farm, Oriole",47,47,47,47,47,47
Glencairn,37,37,37,37,37,37
Hillcrest Village,27,27,27,27,27,27
Humber Summit,11,11,11,11,11,11


#### Let's find out how many unique categories can be curated from all the returned venues


In [25]:
print('There are {} uniques categories.'.format(len(EBC_venues['Venue Category'].unique())))

There are 167 uniques categories.


<a id='item3'></a>


## Analyze Each Neighborhood


In [26]:
# one hot encoding
EBC_onehot = pd.get_dummies(EBC_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
EBC_onehot['Neighborhood'] = EBC_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [EBC_onehot.columns[-1]] + list(EBC_onehot.columns[:-1])
EBC_onehot = EBC_onehot[fixed_columns]

EBC_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food & Drink Shop,Food Court,Food Stand,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Liquor Store,Locksmith,Lounge,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,New American Restaurant,Optical Shop,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Ski Chalet,Smoothie Shop,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,

And let's examine the new dataframe size.


In [27]:
EBC_onehot.shape

(762, 168)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [28]:
EBC_grouped = EBC_onehot.groupby('Neighborhood').mean().reset_index()
EBC_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food & Drink Shop,Food Court,Food Stand,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Liquor Store,Locksmith,Lounge,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,New American Restaurant,Optical Shop,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Ski Chalet,Smoothie Shop,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.00,0.03125,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.062500,0.000000,0.03125,0.031250,0.000000,0.00,0.000000,0.03125,0.000000,0.000000,0.000000,0.031250,0.000000,0.031250,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.031250,0.03125,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.031250,0.000000,0.000000,0.000000,0.031250,0.00,0.03125,0.000000,0.000000,0.000000,0.000000,0.03125,0.000000,0.000000,0.031250,0.000000,0.031250,0.000000,0.031250,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.031250,0.00,0.00000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.03125,0.03125,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.031250,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

#### Let's confirm the new size


In [29]:
EBC_grouped.shape

(20, 168)

In [30]:
num_top_venues = 5

for hood in EBC_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = EBC_grouped[EBC_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0           Bank  0.06
1    Coffee Shop  0.06
2  Deli / Bodega  0.03
3       Bus Line  0.03
4    Bridal Shop  0.03


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.15
1          Gas Station  0.15
2                 Bank  0.15
3           Restaurant  0.08
4                 Park  0.08


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.06
2    Sushi Restaurant  0.04
3      Sandwich Place  0.04
4         Pizza Place  0.04


----Don Mills----
                 venue  freq
0          Coffee Shop  0.08
1          Pizza Place  0.05
2           Restaurant  0.05
3  Japanese Restaurant  0.04
4                  Gym  0.04


----Downsview----
                   venue  freq
0            Coffee Shop  0.09
1  Vietnamese Restaurant  0.07
2                  Hotel  0.06
3            Pizza Place  0.04
4          Grocery Store 

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = EBC_grouped['Neighborhood']

for ind in np.arange(EBC_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(EBC_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Ski Chalet,Diner,Building
1,Bayview Village,Gas Station,Japanese Restaurant,Bank,Skating Rink,Restaurant
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Bagel Shop,Bakery,Sandwich Place
3,Don Mills,Coffee Shop,Pizza Place,Restaurant,Gym,Japanese Restaurant
4,Downsview,Coffee Shop,Vietnamese Restaurant,Hotel,Grocery Store,Gas Station


## Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 4 clusters.


In [33]:
# set number of clusters
kclusters = 7

EBC_grouped_clustering = EBC_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(EBC_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 6, 4, 4, 0, 0, 0, 5, 2, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.


In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

EBC_merged = EBC

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
EBC_merged = EBC_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

EBC_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Bus Stop,Convenience Store,Pharmacy,Shopping Mall
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Intersection,Pizza Place,Middle Eastern Restaurant,Sandwich Place
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Restaurant,Clothing Store,Coffee Shop,Fast Food Restaurant,Dessert Shop
3,M3B,North York,Don Mills,43.745906,-79.352188,4,Coffee Shop,Pizza Place,Restaurant,Gym,Japanese Restaurant
4,M6B,North York,Glencairn,43.709577,-79.445073,0,Fast Food Restaurant,Grocery Store,Coffee Shop,Pizza Place,Restaurant


Finally, let's visualize the resulting clusters


In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(EBC_merged['Latitude'], EBC_merged['Longitude'], EBC_merged['Neighbourhood'], EBC_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [36]:
EBC_merged.loc[EBC_merged['Cluster Labels'] == 0, EBC_merged.columns[[2] + list(range(5, EBC_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Parkwoods,0,Park,Bus Stop,Convenience Store,Pharmacy,Shopping Mall
1,Victoria Village,0,Coffee Shop,Intersection,Pizza Place,Middle Eastern Restaurant,Sandwich Place
2,"Lawrence Manor, Lawrence Heights",0,Restaurant,Clothing Store,Coffee Shop,Fast Food Restaurant,Dessert Shop
4,Glencairn,0,Fast Food Restaurant,Grocery Store,Coffee Shop,Pizza Place,Restaurant
8,"Fairview, Henry Farm, Oriole",0,Coffee Shop,Clothing Store,Restaurant,Juice Bar,Japanese Restaurant
9,"Northwood Park, York University",0,Pizza Place,Coffee Shop,Furniture / Home Store,American Restaurant,Restaurant
11,Downsview,0,Coffee Shop,Vietnamese Restaurant,Hotel,Grocery Store,Gas Station
13,Downsview,0,Coffee Shop,Vietnamese Restaurant,Hotel,Grocery Store,Gas Station
14,"North Park, Maple Leaf Park, Upwood Park",0,Coffee Shop,Dim Sum Restaurant,Park,Convenience Store,Supermarket
17,Downsview,0,Coffee Shop,Vietnamese Restaurant,Hotel,Grocery Store,Gas Station


#### Cluster 2


In [37]:
EBC_merged.loc[EBC_merged['Cluster Labels'] == 1, EBC_merged.columns[[2] + list(range(5, EBC_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
19,"Humberlea, Emery",1,Intersection,Bakery,Coffee Shop,Discount Store,Park


#### Cluster 3


In [38]:
EBC_merged.loc[EBC_merged['Cluster Labels'] == 2, EBC_merged.columns[[2] + list(range(5, EBC_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
15,Humber Summit,2,Electronics Store,Park,Bank,Italian Restaurant,Pizza Place


#### Cluster 4


In [39]:
EBC_merged.loc[EBC_merged['Cluster Labels'] == 3, EBC_merged.columns[[2] + list(range(5, EBC_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,"York Mills, Silver Hills",3,Park,Intersection,Coffee Shop,Gym / Fitness Center,Doner Restaurant


#### Cluster 5


In [40]:
EBC_merged.loc[EBC_merged['Cluster Labels'] == 4, EBC_merged.columns[[2] + list(range(5, EBC_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Don Mills,4,Coffee Shop,Pizza Place,Restaurant,Gym,Japanese Restaurant
5,Don Mills,4,Coffee Shop,Pizza Place,Restaurant,Gym,Japanese Restaurant
7,"Bathurst Manor, Wilson Heights, Downsview North",4,Coffee Shop,Bank,Ski Chalet,Diner,Building
16,"Willowdale, Newtonbrook",4,Korean Restaurant,Café,Coffee Shop,Middle Eastern Restaurant,Pizza Place
18,"Bedford Park, Lawrence Manor East",4,Coffee Shop,Italian Restaurant,Bagel Shop,Bakery,Sandwich Place
20,"Willowdale, Willowdale East",4,Coffee Shop,Japanese Restaurant,Korean Restaurant,Ramen Restaurant,Bubble Tea Shop


#### Cluster 6


In [41]:
EBC_merged.loc[EBC_merged['Cluster Labels'] == 5, EBC_merged.columns[[2] + list(range(5, EBC_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,Hillcrest Village,5,Pharmacy,Park,Bank,Coffee Shop,Grocery Store
23,"Willowdale, Willowdale West",5,Park,Coffee Shop,Grocery Store,Pharmacy,Sandwich Place


#### Cluster 7


In [42]:
EBC_merged.loc[EBC_merged['Cluster Labels'] == 6, EBC_merged.columns[[2] + list(range(5, EBC_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,Bayview Village,6,Gas Station,Japanese Restaurant,Bank,Skating Rink,Restaurant
